<a href="https://colab.research.google.com/github/fatemafaria142/Exploration-of-Large-Language-Models-in-Mental-Health-Advice-Generation/blob/main/Mental_Health_Advice_Generation_using_Llama_2_7b_chat_hf_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

#### **Dataset Link:** https://huggingface.co/datasets/Amod/mental_health_counseling_conversations

In [3]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("Amod/mental_health_counseling_conversations")

In [4]:
# Select 500 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(500))

# Select 200 rows for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(500, 700))


In [5]:
train_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 500
})

In [6]:
test_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 200
})

In [7]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 500
Test subset size: 200


In [8]:
# Show 5 examples from the train dataset
print("Train subset examples:")
for i in range(5):
    context = train_subset['Context'][i]
    response = train_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")

Train subset examples:
Example 1:
Context: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Response: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves

In [9]:
# Show 5 examples from the test dataset
print("\nTest subset examples:")
for i in range(5):
    context = test_subset['Context'][i]
    response = test_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")


Test subset examples:
Example 1:
Context: I never feel like myself. I can’t even think straight anymore. I start stuttering and I can’t remember anything. I always get nervous and usually talk myself down but recently end up fighting with, what feels like, someone else. I don’t know why I feel this way, but I hate it.
Response: Hi Houston, Okay, something's definitely going on, right? My gut is to start with your family physician, because you have some physical symptoms. While it seems anxiety is a likely problem here, your doctor will hopefully know your history and can help you decide if you need some medical tests or some therapy. Without knowing more, that's all I can say at this point. Best of luck.
-------------------
Example 2:
Context: I never feel like myself. I can’t even think straight anymore. I start stuttering and I can’t remember anything. I always get nervous and usually talk myself down but recently end up fighting with, what feels like, someone else. I don’t know why

#### **Creating a Mental Health Support Prompt Using LangChain**

In [10]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Creating prompt for large language model
    pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n"""
    prompt = pre_prompt + "Here is my mental health problems: {context}" +"[\INST]"+"\nCounselor's Advice: {response}"

    # Extract context and response from the sample
    example_context = sample['Context']
    example_response = sample['Response']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=["context", "response"])
    single_turn_prompt = prompt_template.format(context=example_context, response=example_response)

    return single_turn_prompt


#### **Generating LangChain Prompt for a Training Subset Examples**

In [11]:
prompt = create_prompt(train_subset[0])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?[\INST]
Counselor's Advice: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be kno

In [12]:
prompt = create_prompt(train_subset[1])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?[\INST]
Counselor's Advice: Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone. Changing our feelings is like changing our thoughts - it's hard to do. Our mi

In [13]:
prompt = create_prompt(train_subset[2])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?[\INST]
Counselor's Advice: First thing I'd suggest is getting the sleep you need or it will impact how you think and feel. I'd look at finding what is going well in your life and what you can be grateful for. I believe everyone has talents and wants to find their purpose in life. I think you can figure it out with some help.


#### **Generating LangChain Prompt for a Testing Subset Examples**

In [14]:
prompt = create_prompt(test_subset[0])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I never feel like myself. I can’t even think straight anymore. I start stuttering and I can’t remember anything. I always get nervous and usually talk myself down but recently end up fighting with, what feels like, someone else. I don’t know why I feel this way, but I hate it.[\INST]
Counselor's Advice: Hi Houston, Okay, something's definitely going on, right? My gut is to start with your family physician, because you have some physical symptoms. While it seems anxiety is a likely problem here, your doctor will hopefully know your history and can help you decide if you need some medical tests or some therapy. Without knowing more, that's all I can say at 

In [15]:
prompt = create_prompt(test_subset[1])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I never feel like myself. I can’t even think straight anymore. I start stuttering and I can’t remember anything. I always get nervous and usually talk myself down but recently end up fighting with, what feels like, someone else. I don’t know why I feel this way, but I hate it.[\INST]
Counselor's Advice: What you are describing sounds like it may be a form of dissociation. Dissociation is our mind's way of disconnecting us from aspects of our experience in an attempt to protect us in overwhelming situations. It also sounds like you are noticing anxiety in certain situations. In working with a competent therapist, you may be able to gain insight into these 

In [16]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I never feel like myself. I can’t even think straight anymore. I start stuttering and I can’t remember anything. I always get nervous and usually talk myself down but recently end up fighting with, what feels like, someone else. I don’t know why I feel this way, but I hate it.[\INST]
Counselor's Advice: Hello,I am wondering if you might be experiencing some form of dissociation.  People who dissociate might feel like the world around them is unreal and unfamiliar, or that they are not really in their bodies and that they are observing themselves from outside of their bodies.  Dissociation can also mean that there are parts of you that have "split off" and

In [17]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I never feel like myself. I can’t even think straight anymore. I start stuttering and I can’t remember anything. I always get nervous and usually talk myself down but recently end up fighting with, what feels like, someone else. I don’t know why I feel this way, but I hate it.[\INST]
Counselor's Advice: Hello,I am wondering if you might be experiencing some form of dissociation.  People who dissociate might feel like the world around them is unreal and unfamiliar, or that they are not really in their bodies and that they are observing themselves from outside of their bodies.  Dissociation can also mean that there are parts of you that have "split off" and

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [20]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [21]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [22]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [23]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [24]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.8  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id,  top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)


# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n"""
prompt = pre_prompt + "Here is my mental health problems: {context}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["context"])


chain = prompt | llm

context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

print(chain.invoke({"context": context}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Thank you for sharing your struggles with me. It takes a lot of courage to open up about one's feelings, especially when it comes to sensitive topics like mental health. I want you to know that you are not alone in this struggle, and there are many people who have gone through similar experiences.

Firstly, it's important to recognize that these feelings of worthlessness and negative self-talk are not uncommon. Many people struggle with these types of thoughts, and it doesn't mean that there is anything inherently wrong with you. It's important to remember that these thoughts are just thoughts, and they don't define your worth as a person.

One way to start working on these feelings is to practice self-compassion. This means treating yourself with kindness, understanding, and acceptance, just as you would treat a close friend. Instead of beating yourself up with negative self-talk, try offering yourself support and encouragement. Remind yourself that you are doing the best you can, a

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [25]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [26]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [27]:
from transformers import TrainingArguments
output_model= "Llama_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [28]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [29]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.146000
20,1.812700
30,1.790200
40,1.604100
50,1.569600
60,1.576100
70,1.574500
80,1.565300
90,1.508700
100,1.486400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=150, training_loss=1.6138535308837891, metrics={'train_runtime': 2300.2036, 'train_samples_per_second': 0.261, 'train_steps_per_second': 0.065, 'total_flos': 2.4372773388288e+16, 'train_loss': 1.6138535308837891, 'epoch': 3.14})

### **Save the model**

In [30]:
trainer.save_model("Llama_instruct_generation")

In [31]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

# **Conversation Buffer memory**

In [32]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n"""
prompt = pre_prompt + "Here is my mental health problems: {context}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["context"])

In [33]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [34]:
context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Hello there, thank you for sharing your struggles with me. It takes a lot of courage to open up about one\'s feelings, and I\'m here to offer you a safe and non-judgmental space to talk about them.\n\nFirstly, please know that you are not alone in this struggle. Many people experience feelings of worthlessness and low self-esteem, and it\'s important to remember that these feelings do not define your worth as a person. You are more than just your thoughts and feelings, and you have so much to offer the world.\n\nIt\'s great to hear that you haven\'t contemplated or tried suicide. That shows a lot of resilience and hope, and it\'s important to hold onto that. It\'s okay to feel overwhelmed and hopeless at times, but there are many people who care about you and want to help you through these tough times.\n\nNow, let\'s talk about some strategies that can help you manage these feelings of worthlessness. One important thing is to practice self-compassion. This means treating yourself wi

#### **Example: 02**

In [35]:
context = "I'm 15 and my girlfriend is 14. Am I a pedophile because I'm attracted to her and she's under 18?"
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I'm 15 and my girlfriend is 14. Am I a pedophile because I'm attracted to her and she's under 18?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  As a mental health counselor, I must inform you that it is not appropriate or healthy to be sexually attracted to a 14-year-old, regardless of their relationship with you. It is important to understand that engaging in any form of sexual activity with a minor is illegal and can have serious legal and social consequences.\n\nIt is important to recognize that sexual attraction to a minor is not a normal or healthy part of a romantic relationship, and it can cause significant harm to both parties involved. It is important to prioritize the well-being and safety of the minor, and to seek professional help to address any issues related to your own sexual attraction.\n\nI strongly advise you to seek professional help from a licensed therapist or counselor who specializes in treating individuals with these types of attractions. They can provide you with the support and guidance you need to work through these issues in a safe and responsible manner.\n\nRemember, it is important to prioritiz

#### **Example: 03**

In [36]:
context = "I'm always listening to my husband, but it feels like he never listens to me."
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I'm always listening to my husband, but it feels like he never listens to me.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Hello there, thank you for sharing your concerns with me. It takes a lot of courage to reach out for help, and I\'m here to offer you support and guidance. It sounds like you\'re feeling really frustrated and misunderstood in your relationship with your husband.\n\nFirst of all, it\'s important to recognize that it\'s not uncommon for people to feel like they\'re not being heard or understood in their relationships. It can be especially challenging when it feels like you\'re always the one trying to communicate and your partner is not listening.\n\nCan you tell me more about what it\'s like to feel like your husband is not listening to you? What are some specific examples of situations where you feel like you\'re not being heard?\n\nIt\'s important to remember that communication is a two-way street, and it\'s possible that your husband may be feeling just as unheard and misunderstood as you are. It might be helpful to have an open and honest conversation with him about how you\'re f

In [37]:
memory.load_memory_variables({})

{'history': 'Human: I\'m going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I\'m worthless and how I shouldn\'t be here.\n   I\'ve never tried or contemplated suicide. I\'ve always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?\nAI:   Hello there, thank you for sharing your struggles with me. It takes a lot of courage to open up about one\'s feelings, and I\'m here to offer you a safe and non-judgmental space to talk about them.\n\nFirstly, please know that you are not alone in this struggle. Many people experience feelings of worthlessness and low self-esteem, and it\'s important to remember that these feelings do not define your worth as a person. You are more than just your thoughts and feelings, and you have so much to offer the world.\n\nIt\'s great to hear that you haven\'t contemplated or tried suicide. That shows a lot of resilience and hope, and it\'s imp

In [38]:
print(memory.buffer)

Human: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?
AI:   Hello there, thank you for sharing your struggles with me. It takes a lot of courage to open up about one's feelings, and I'm here to offer you a safe and non-judgmental space to talk about them.

Firstly, please know that you are not alone in this struggle. Many people experience feelings of worthlessness and low self-esteem, and it's important to remember that these feelings do not define your worth as a person. You are more than just your thoughts and feelings, and you have so much to offer the world.

It's great to hear that you haven't contemplated or tried suicide. That shows a lot of resilience and hope, and it's important to hold onto that. It's

# **Conversation Buffer Window Memory**
* ConversationBufferWindowMemory keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [39]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n"""
prompt = pre_prompt + "Here is my mental health problems: {context}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["context"])

In [41]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferWindowMemory(k=2)
chat_llm_chain_window = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [42]:
context = "I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on."
chat_llm_chain_window.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  As a thoughtful mental health counselor, I can understand how difficult this situation must be for you. Losing a child to overdose is a traumatic experience, and the added layer of blame and guilt from your daughter can make it even more challenging to cope with.\n\nFirstly, please know that you are not alone in this struggle. Many parents have had to deal with the loss of a child to drug addiction, and it's important to remember that you don't have to carry this burden alone. Reaching out for support from friends, family, or a mental health professional can be incredibly helpful in navigating this difficult time.\n\nIt's important to recognize that addiction is a complex issue that affects not only the individual struggling with it but also their loved ones. It's not uncommon for family members to feel guilt, shame, or anger towards each other during this process. However, it's essential to separate the addiction from the individual and to avoid blaming each other.\n\nInstead of fo

#### **Example: 02**

In [43]:
context = "I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?"
chat_llm_chain_window.predict(context=context)




> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Thank you for reaching out and sharing your concerns with me. It takes a lot of courage to open up about personal struggles, and I'm here to offer you a safe and non-judgmental space to talk about your feelings.\n\nFirstly, it's important to acknowledge that it's completely normal to feel upset or conflicted when spending time with someone you have feelings for, especially if it's someone you were in a relationship with in the past. It's understandable that you may be feeling uncertain or guilty about the situation, and it's important to give yourself time and space to process these emotions.\n\nIt's great that you're looking for support and guidance on how to navigate this situation. It's important to remember that you don't have to go through this alone, and there are many resources available to help you work through your feelings.\n\nCan you tell me more about what happened over the weekend? What were some of the things that made you feel upset or guilty, and how did your boyfrie

#### **Example: 03**

In [44]:
context = "I'm depressed. I have been for years. I hide it from everyone because I'm scared of the reactions I'll get. Last time I tried telling my parents, it was a huge argument about me being too young to be depressed (I’m a legal adult), calling me ungrateful, and telling me that if I can't handle things now, it's only going to get worse in the future (which is turning out to be true). It's exhausting pretending to be okay, and I don't know how much longer I can try. I'm just really tired, and sadly, I can't afford the help I need on my own."
chat_llm_chain_window.predict(context=context)




> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.
Here is my mental health problems: I'm depressed. I have been for years. I hide it from everyone because I'm scared of the reactions I'll get. Last time I tried telling my parents, it was a huge argument about me being too young to be depressed (I’m a legal adult), calling me ungrateful, and telling me that if I can't handle things now, it's only going to get worse in the future (which is turning out to be true). It's exhausting pretending to be okay, and I don't know how much longer I can try. I'm just really tired, and sadly, I can't afford the help I need on my own.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Hello there, thank you for reaching out and sharing your struggles with me. It takes a lot of courage to open up about your feelings, especially when it comes to mental health. I want you to know that you are not alone in this, and there are people who care about you and want to help.\n\nFirst of all, please know that depression is a common and treatable condition. It\'s not something that you can simply "snap out of," but it is something that can be managed with the right support and treatment. It\'s important to remember that you don\'t have to go through this alone, and there are many resources available to help you.\n\nIt\'s understandable that you may be hesitant to reach out to your parents, especially after the argument you had with them last time. However, it\'s important to remember that they are your parents and they want the best for you. They may not fully understand what you\'re going through, but they can offer you support and help you find the resources you need.\n\nI

In [45]:
memory.load_memory_variables({})

{'history': 'Human: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?\nAI:   Thank you for reaching out and sharing your concerns with me. It takes a lot of courage to open up about personal struggles, and I\'m here to offer you a safe and non-judgmental space to talk about your feelings.\n\nFirstly, it\'s important to acknowledge that it\'s completely normal to feel upset or conflicted when spending time with someone you have feelings for, especially if it\'s someone you were in a relationship with in the past. It\'s understandable that you may be feeling uncertain or guilty about the situation, and it\'s important to give yourself time and space to process these emotions.\n\nIt\'s great that you\'re looking for support and guidance on how to navigate this situation. It\'s important to remember that you don\'t have to go through this alone, and the

In [46]:
print(memory.buffer)

Human: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?
AI:   Thank you for reaching out and sharing your concerns with me. It takes a lot of courage to open up about personal struggles, and I'm here to offer you a safe and non-judgmental space to talk about your feelings.

Firstly, it's important to acknowledge that it's completely normal to feel upset or conflicted when spending time with someone you have feelings for, especially if it's someone you were in a relationship with in the past. It's understandable that you may be feeling uncertain or guilty about the situation, and it's important to give yourself time and space to process these emotions.

It's great that you're looking for support and guidance on how to navigate this situation. It's important to remember that you don't have to go through this alone, and there are many resources availa